In [1]:
import pandas as pd
import numpy as np
import os

# Correct base path
BASE_PATH = "E:/EcopackAI/data"

# Load ORIGINAL dataset (Excel file)
materials = pd.read_excel(os.path.join(BASE_PATH, "packaging_materials.xlsx"))

print("Dataset Shape:", materials.shape)
print("\nColumns:\n", materials.columns.tolist())

materials.head()

Dataset Shape: (9000, 30)

Columns:
 ['Material_ID', 'Material_Type', 'Tensile_Strength_MPa', 'Weight_Capacity_kg', 'Thickness_Micrometers', 'Production_Cost_per_kg_INR', 'Shelf_Life_Days', 'Biodegradability_Score', 'CO2_Emission_Score', 'Recyclability_Percentage', 'Moisture_Barrier_g_m2_day', 'Oxygen_Barrier_cc_m2_day', 'Processing_Temperature_C', 'Density_g_cm3', 'Biodegradation_Time_Days', 'Water_Absorption_Percentage', 'Thermal_Stability_C', 'Environmental_Impact_Score', 'Application_Type', 'Manufacturing_Method', 'Additives_Used', 'Supplier_Region_India', 'Food_Product_Type', 'Storage_Temperature', 'Certification', 'Market_State', 'Barrier_Property', 'Testing_Date', 'GST_Applicable', 'Indian_Standard']


,Material_ID,Material_Type,Tensile_Strength_MPa,Weight_Capacity_kg,Thickness_Micrometers,Production_Cost_per_kg_INR,Shelf_Life_Days,Biodegradability_Score,CO2_Emission_Score,Recyclability_Percentage,...,Additives_Used,Supplier_Region_India,Food_Product_Type,Storage_Temperature,Certification,Market_State,Barrier_Property,Testing_Date,GST_Applicable,Indian_Standard
0,IND_MAT_00001,Alginate,39.94,116.87,228.10,669.99,90,83.17,93.80,94.33,...,Glycerol,East India,Organic Products,Room Temperature (20-25°C),Emerging Certification,New Product,High Oxygen Barrier,2025-07-02 12:08:02.243,Yes (12%),NaN
1,IND_MAT_00002,Alginate,90.95,75.88,133.10,1185.69,120,71.61,55.40,96.61,...,NaN,Central India,Dairy Products,Room Temperature (20-25°C),FSSAI Approved,Mature,High Gas Barrier,2025-08-31 12:08:02.243,Yes (12%),BIS
2,IND_MAT_00003,Polybutylene Succinate (PBS),71.58,96.95,38.80,719.21,180,74.92,95.92,32.30,...,Citric Acid,Central India,Rice,Ambient (25-30°C),ISI Certified,New Product,High Moisture Barrier,2025-02-06 12:08:02.243,Yes (18%),ISI
3,IND_MAT_00004,Seaweed/Kelp-based Polymer,59.78,66.45,194.22,337.05,365,86.54,68.20,53.56,...,Natural Extracts,South India,Edible Oils,Room Temperature (20-25°C),Emerging Certification,Growth,High Oxygen Barrier,2024-08-26 12:08:02.243,Yes (12%),ISI
4,IND_MAT_00005,Blended Starch,20.59,86.94,91.53,480.16,30,94.12,77.48,88.94,...,Glycerol,East India,Snacks (Namkeen),Room Temperature (20-25°C),EN 13432,New Product,High Oxygen Barrier,2025-11-08 12:08:02.243,Yes (12%),ISI


In [2]:
# Columns actually useful for analytics & charts
required_cols = [
    "Material_Type",
    "Tensile_Strength_MPa",
    "Weight_Capacity_kg",
    "Thickness_Micrometers",
    "Thermal_Stability_C",
    "Biodegradability_Score",
    "Recyclability_Percentage",
    "CO2_Emission_Score",
    "Environmental_Impact_Score",
    "Density_g_cm3",
    "Moisture_Barrier_g_m2_day",
    "Oxygen_Barrier_cc_m2_day"
]

materials_clean = materials[required_cols].copy()

print("New shape:", materials_clean.shape)
materials_clean.head()

New shape: (9000, 12)


,Material_Type,Tensile_Strength_MPa,Weight_Capacity_kg,Thickness_Micrometers,Thermal_Stability_C,Biodegradability_Score,Recyclability_Percentage,CO2_Emission_Score,Environmental_Impact_Score,Density_g_cm3,Moisture_Barrier_g_m2_day,Oxygen_Barrier_cc_m2_day
0,Alginate,39.94,116.87,228.10,168.01,83.17,94.33,93.80,24.33,0.93,51.55,193.82
1,Alginate,90.95,75.88,133.10,233.24,71.61,96.61,55.40,79.66,1.08,109.38,250.00
2,Polybutylene Succinate (PBS),71.58,96.95,38.80,162.94,74.92,32.30,95.92,51.16,1.22,32.41,58.31
3,Seaweed/Kelp-based Polymer,59.78,66.45,194.22,212.45,86.54,53.56,68.20,25.44,1.00,30.01,277.88
4,Blended Starch,20.59,86.94,91.53,230.32,94.12,88.94,77.48,63.90,1.30,82.18,90.92


In [3]:
# -------- Normalization Helper --------
def normalize(col):
    return (col - col.min()) / (col.max() - col.min() + 1e-6)

# -------- Protection Score --------
materials_clean["Protection_Score"] = (
    0.35 * normalize(materials_clean["Tensile_Strength_MPa"]) +
    0.25 * normalize(materials_clean["Weight_Capacity_kg"]) +
    0.20 * normalize(materials_clean["Thickness_Micrometers"]) +
    0.20 * normalize(materials_clean["Thermal_Stability_C"])
)

# -------- Sustainability Score --------
materials_clean["Sustainability_Score"] = (
    0.30 * normalize(materials_clean["Biodegradability_Score"]) +
    0.25 * normalize(materials_clean["Recyclability_Percentage"]) +
    0.25 * (1 - normalize(materials_clean["CO2_Emission_Score"])) +
    0.20 * (1 - normalize(materials_clean["Environmental_Impact_Score"]))
)

# -------- Material Efficiency --------
materials_clean["Material_Efficiency"] = (
    0.6 * materials_clean["Protection_Score"] +
    0.4 * materials_clean["Sustainability_Score"]
)

# -------- EcoScore (Dashboard KPI Score) --------
materials_clean["EcoScore"] = (
    0.25 * normalize(materials_clean["Biodegradability_Score"]) +
    0.20 * normalize(materials_clean["Recyclability_Percentage"]) +
    0.20 * (1 - normalize(materials_clean["CO2_Emission_Score"])) +
    0.20 * (1 - normalize(materials_clean["Environmental_Impact_Score"])) +
    0.15 * normalize(materials_clean["Tensile_Strength_MPa"])
)

materials_clean.head()

,Material_Type,Tensile_Strength_MPa,Weight_Capacity_kg,Thickness_Micrometers,Thermal_Stability_C,Biodegradability_Score,Recyclability_Percentage,CO2_Emission_Score,Environmental_Impact_Score,Density_g_cm3,Moisture_Barrier_g_m2_day,Oxygen_Barrier_cc_m2_day,Protection_Score,Sustainability_Score,Material_Efficiency,EcoScore
0,Alginate,39.94,116.87,228.10,168.01,83.17,94.33,93.80,24.33,0.93,51.55,193.82,0.576932,0.645182,0.604232,0.616505
1,Alginate,90.95,75.88,133.10,233.24,71.61,96.61,55.40,79.66,1.08,109.38,250.00,0.731164,0.642582,0.695731,0.668914
2,Polybutylene Succinate (PBS),71.58,96.95,38.80,162.94,74.92,32.30,95.92,51.16,1.22,32.41,58.31,0.483588,0.281866,0.402899,0.363420
3,Seaweed/Kelp-based Polymer,59.78,66.45,194.22,212.45,86.54,53.56,68.20,25.44,1.00,30.01,277.88,0.596025,0.653384,0.618969,0.656807
4,Blended Starch,20.59,86.94,91.53,230.32,94.12,88.94,77.48,63.90,1.30,82.18,90.92,0.437898,0.678050,0.533959,0.591197


In [4]:
# Group by material and take mean values
material_summary = (
    materials_clean
    .groupby("Material_Type", as_index=False)
    .agg({
        "Tensile_Strength_MPa":"mean",
        "Weight_Capacity_kg":"mean",
        "Thickness_Micrometers":"mean",
        "Thermal_Stability_C":"mean",
        "Biodegradability_Score":"mean",
        "Recyclability_Percentage":"mean",
        "CO2_Emission_Score":"mean",
        "Environmental_Impact_Score":"mean",
        "Density_g_cm3":"mean",
        "Moisture_Barrier_g_m2_day":"mean",
        "Oxygen_Barrier_cc_m2_day":"mean",
        "Protection_Score":"mean",
        "Sustainability_Score":"mean",
        "Material_Efficiency":"mean",
        "EcoScore":"mean"
    })
)

print("Unique materials:", material_summary.shape[0])
material_summary.head()

Unique materials: 31


,Material_Type,Tensile_Strength_MPa,Weight_Capacity_kg,Thickness_Micrometers,Thermal_Stability_C,Biodegradability_Score,Recyclability_Percentage,CO2_Emission_Score,Environmental_Impact_Score,Density_g_cm3,Moisture_Barrier_g_m2_day,Oxygen_Barrier_cc_m2_day,Protection_Score,Sustainability_Score,Material_Efficiency,EcoScore
0,Alginate,50.051179,74.255286,126.903286,200.265607,74.746679,65.038607,78.467964,57.374500,1.156000,76.923750,152.170500,0.494364,0.479836,0.488553,0.481958
1,Bagasse Fiber Composite,51.460606,72.804377,129.093973,199.363232,76.121313,67.566364,76.488283,58.061178,1.149192,77.417340,143.656835,0.496561,0.506788,0.500652,0.505830
2,Blended Starch,50.224130,74.928703,133.360171,200.372662,76.565358,65.605631,76.845597,57.644608,1.152253,74.713959,147.493959,0.502353,0.501544,0.502030,0.499856
3,Cellulose Acetate,47.264319,75.594884,136.032359,200.568538,77.133056,63.381030,77.707741,57.211130,1.138239,78.996379,155.302691,0.494893,0.493401,0.494296,0.488676
4,Chitosan,50.147491,73.079068,131.335054,200.451792,78.483584,66.144803,76.483799,56.736918,1.142545,75.572186,150.439391,0.496104,0.520039,0.505678,0.515410


In [5]:
output_path = "E:/EcopackAI/models/dashboard_materials.csv"

material_summary.to_csv(output_path, index=False)

print("Dashboard dataset saved successfully!")
print("Location:", output_path)

Dashboard dataset saved successfully!
Location: E:/EcopackAI/models/dashboard_materials.csv
